In [1]:
import numpy as np
import pandas as pd 

In [2]:
eventCodes = pd.read_csv('Event_Codes.txt', sep='\t')
#Strip excess whitespace that I noticed in Event Message Type Description
eventCodes['Event_Msg_Type_Description'] = eventCodes['Event_Msg_Type_Description'].map(str.strip)

GameLineup = pd.read_csv('Game_Lineup.txt', sep='\t')
PlayByPlay = pd.read_csv('Play_by_Play.txt', sep='\t')
GameIDs = PlayByPlay['Game_id'].unique()

In [3]:
def sortedPlayByPlay(PlayByPlay):
    """Returns PlayByPlay with nested sorted given by hackathon pdf file.
    
    Input:
        PlayByPlay: Pandas Dataframe from reading Play_by_Play csv file"""
    return PlayByPlay.sort_values(['Period', 'PC_Time', 'WC_Time', 'Event_Num'], ascending=[True, False, True,True])

In [4]:
def readablePlayByPlay(playByPlay, gameLineup, eventCodes):
    """Maps teamIDs to correct player and replaces it with a simple team0 or team1. 
    Also maps event numbers and action to corresponding description, based off eventCodes"""
    gameTeams = list(gameLineup['Team_id'].unique())
    playerList = list(gameLineup['Person_id'].unique())
    
    playByPlay['Event_Msg_Type'] = playByPlay['Event_Msg_Type'].astype(str)
    playByPlay['Action_Type'] = playByPlay['Action_Type'].astype(str)
    
    print('Team0', gameTeams[0])
    print('Team1', gameTeams[1])
    
    for event in playByPlay.itertuples():
        i = getattr(event, 'Index')
        player = getattr(event, 'Person1')
        teamID = getattr(event, 'Team_id')
        
        if teamID not in gameTeams:
            playerTeam = 'Officials'
        else:
            playerTeam = getPlayerTeam(player, teamID, playerList, gameLineup)
        #print(playerTeam)
        #print(gameTeams)
        
        if playerTeam == gameTeams[0]:
            playByPlay.at[i,'Team_id'] = 'Team0'
        elif playerTeam == gameTeams[1]:
            playByPlay.at[i, 'Team_id'] = 'Team1'
        else:
            playByPlay.at[i, 'Team_id'] = 'Officials'

        
        eventNum = getattr(event, 'Event_Num')
         #Get the event type
        eventMsgType = int(getattr(event, "Event_Msg_Type"))
        actionType = int(getattr(event, 'Action_Type'))
        
        #Turn the event type and action type into a description
        #First find corresponding row in eventCodes file
        eventCode = eventCodes[(eventCodes['Event_Msg_Type'] == eventMsgType) & (eventCodes['Action_Type'] == actionType)]

        #Description should be unique based off ID, so just get first element of list created
        eventMsgDescript = list(eventCode['Event_Msg_Type_Description'].unique())[0]
        actionTypeDescript = list(eventCode['Action_Type_Description'].unique())[0]
        
        playByPlay.at[i, 'Event_Msg_Type'] = eventMsgDescript
        playByPlay.at[i, 'Action_Type'] = actionTypeDescript
    return playByPlay

In [5]:
def addPoints(player, points, team0Lineup, team1Lineup, gameDict, possession):
    #Player is person1 of the event messages. Every teammate gets the points for added and opponents points against
    if player in team0Lineup:
        for team0player in team0Lineup:
            #Add points for and off possession
            gameDict[team0player] += np.array([points, 0, possession, 0])
        for team1player in team1Lineup:
            #Add points against and def posession
            gameDict[team1player] += np.array([0, points, 0, possession])
        return
    elif player in team1Lineup:
        for team0player in team0Lineup:
            #Add points against and def possession
            gameDict[team0player] += np.array([0, points, 0, possession])
        for team1player in team1Lineup:
            #Add points for and off possession
            gameDict[team1player] += np.array([points, 0, possession, 0])
        return
    else:
        print("Player not found on either team lineup!")
        print(player)
        print('Team0', team0Lineup)
        print('Team1', team1Lineup)
        return -1
    
def addPossession(team0Lineup, team1Lineup, gameDict):
    """Adds offensive possession to team0Lineup and defensive possession to team1Lineup."""
    for team0player in team0Lineup:
        #Add off possession
        gameDict[team0player] += np.array([0, 0, 1, 0])
    for team1player in team1Lineup:
        #Add def posession
        gameDict[team1player] += np.array([0, 0, 0, 1])
    return

def getPlayerTeam(player, teamID, playerList, gameLineup):
    """Returns the team of the input player (if player is real) otherwise returns teamID"""
    if player not in playerList:
        #Team event
        playerTeam = teamID
    else:
        playerTeam = gameLineup[gameLineup['Person_id'] == player]['Team_id'].values[0]
    return playerTeam

def doSubstitution(player, event, team0Lineup, team1Lineup, verbose):
    leavingPlayer = player
    enteringPlayer = getattr(event, 'Person2')
    if leavingPlayer in team0Lineup:
        team0Lineup.remove(leavingPlayer)
        team0Lineup.append(enteringPlayer)
    else:
        team1Lineup.remove(leavingPlayer)
        team1Lineup.append(enteringPlayer)
            
    if verbose:
        print(enteringPlayer, ' replacing ', leavingPlayer)

def doPossessionFixSub(possessionInfo, team, gameDict, gameTeams, verbose = False):
    """
    Adds possessions to players given by possessionInfo dictionary. 
        Used for when possession unknown (often due to a team rebound).
        
    
    Inputs: 
        possessionInfo: Dictionary containing the relevant information of the events leading to unknown possession.
            'team0' and 'team1' keys correspond to the team0 and team1 lineups. 
            'event' key corresponds to the event (missed shot or free throw)
            'lastTeam' key corresponds to team last known to have possession
            'subbedPlayers0' and 'subbedPlayers1' keys correspond to players subbed out in between 'event' and 
                play where possession becomes known
            'remainingPlayers0' and 'remainingPlayers1' keys typically correspond to players 
                who are still on court after a substitution occurs ie remainingPlayers = teamLineup - subbedInPlayers
                However, currently assuming that all players on court get credited with possession on missed ft 
                so will just be same as teamLineup
        
        team: Team that currently has possession to be compared to lastTeam
        
        gameDict: Dictionary containing all pointsFor/Against and off/defPossessions of each player to be updated
        
        gameTeams: List of both teams playing to be used to check if team0 or 1 has possession
        
        verbose: Optional input, where if True, prints statements describing which players possession added to
        
        """
    eventDescript = possessionInfo['event']
    team0Lineup = possessionInfo['team0']
    team1Lineup = possessionInfo['team1']
    lastTeam = possessionInfo['lastTeam']
    subbedPlayers0 = possessionInfo['subbedPlayers0']
    subbedPlayers1 = possessionInfo['subbedPlayers1']
    
    possessionInfoEmpty = {'team0': [], 'team1': [], 'remainingPlayers0': [], 'remainingPlayers1': [], 
                           'event': '', 'lastTeam': '', 'subbedPlayers0': [], 'subbedPlayers1': []}
    
    if (team not in gameTeams) or (lastTeam not in gameTeams):
        raise Exception('Either {} or {} not a valid team'.format(team, lastTeam))
    
    if (eventDescript == 'Missed Shot') or (eventDescript == 'Technical Free Throw'):
        if verbose:
            print('Missed shot possession fix')
        if len(subbedPlayers0) + len(subbedPlayers1) > 0:
            #Regardless of who rebounds add possession to players subbed off
            if team == gameTeams[0]:
                #Team0 players who were subbed now on offense (ASSUMES sub happens after rebound)
                addPossession(subbedPlayers0, subbedPlayers1, gameDict)
                if verbose:
                    print("Add off possession to subbed players", subbedPlayers0)
                    print("Add def possession to subbed players", subbedPlayers1)
            else:
                addPossession(subbedPlayers1, subbedPlayers0, gameDict)
                if verbose:
                    print("Add off possession to subbed players", subbedPlayers1)
                    print("Add def possession to subbed players", subbedPlayers0)
        if lastTeam == team:
            #Offensive rebound on missed shot, do nothing except for subbed players
            
            if verbose:
                print("Was offensive rebound, don't do anything for on court players")
            return possessionInfoEmpty
        elif team == gameTeams[0]:
            #Defensive rebound by team0
            addPossession(team1Lineup, team0Lineup, gameDict)
            
            if verbose:
                print("Add off possession to ", team1Lineup)
                print("Add def possession to ", team0Lineup)
        else:
            #Defensive rebound by team1
            addPossession(team0Lineup, team1Lineup, gameDict)

            if verbose:
                print("Add off possession to ", team0Lineup)
                print("Add def possession to ", team1Lineup)
                    
    elif eventDescript == 'Missed Last Free Throw':
        if verbose:
            print('FT possession fix')
        if lastTeam == team:
            #Do nothing, as possession already added to subbed players
            pass
            
        else: #Defensive Rebound
            #Only add possession to those who were on court when foul occured
            ##UPDATE: I believe NBA actually counts it as a possession for all players on court, 
            #so while still labeled remainingPlayers, input in gameRatings function will be teamLineup
            remainingPlayers0 = possessionInfo['remainingPlayers0']
            remainingPlayers1 = possessionInfo['remainingPlayers1']
            
            if team == gameTeams[0]:
                #team0 grabs defensive rebound
                addPossession(remainingPlayers1, remainingPlayers0, gameDict)
                if verbose:
                    print("Add off possession to ", remainingPlayers1)
                    print("Add def possession to ", remainingPlayers0)
            else:
                addPossession(remainingPlayers0, remainingPlayers1, gameDict)
                if verbose:
                    print("Add off possession to ", remainingPlayers0)
                    print("Add def possession to ", remainingPlayers1)
    else:
        raise Exception('Attempt to fix possession failed, invalid event Descript: {}'.format(eventDescript))
    
    return possessionInfoEmpty

In [6]:
def gameRatingsSubCleaned(gamePlayByPlay, gameLineup, eventCodes, verbose = False):
    """Turns a given PlayByPlay log and calculates the number of points and possessions a player is in the game for, 
    both scored by and against the players team. Can be used to calculate offensive and defensive rating defined by 
    offRtg = 100*pointsFor/offPossessions and defRtg = 100*pointsAgainst/defPossesssions
    
    Inputs:
        gamePlayByPlay: A Pandas dataframe object listing the play by play events for a single game. 
            This should be sorted according to the prompt (Period, PC_Time, WC_Time, Event_Num). Taken from Play_by_Play.txt .
        
        gameLineup: A Pandas dataframe of the game lineup for the same gams as the play by play. Taken from Game_Lineup.txt .
        
        eventCodes: Pandas dataframe of the event codes provided by the NBA. Taken from Event_Codes.txt .
        
        verbose: Optional argument. If set to True, prints the current loop counter and a description of each event 
            (made shot, rebound, turnover, etc)
        
    Output:
        gameDict: A dictionary where the keys are each playerID from the gameLineup input, 
            and the value is an array of [pointsFor, pointsAgainst, offPossessions, defPossessions]
    
    """
    
    lastDescript = ''
    counter = 0
    gameTeams = list(gameLineup['Team_id'].unique())
    
    #Create dictionary with keys of each player and value of array([PointsFor, PointsAgainst, OffPossessions, DefPossessions])
    playerList = list(gameLineup['Person_id'].unique())
    gameDict = {player: np.array([0,0,0,0]) for player in playerList}
    
    #List of event numbers to skip 
    #(designed for the case where we 'look ahead' at a future rebound or substitution due to playbyplay error)
    skipEventNums = []
    
    #Keep track of team with possession 
    currPoss = ''
    #Keep track of play clock time (due to weird handling of technical fouls as possessions by NBA) 
    #Initialize to -1 so no accidental equalities
    currTime = -1 
    
    #Added because of cases where made shot with 0 on clock would lead to another possession being added when period ended
    #Unclear if have to take into account other actions at 0
    skipEndPeriod = False
    
    for event in gamePlayByPlay.itertuples():
        
        eventNum = getattr(event, 'Event_Num')
        
         #Get the event type
        eventMsgType = getattr(event, "Event_Msg_Type")
        actionType = getattr(event, 'Action_Type')
        
        #Turn the event type and action type into a description
        #First find corresponding row in eventCodes file
        eventCode = eventCodes[(eventCodes['Event_Msg_Type'] == eventMsgType) & (eventCodes['Action_Type'] == actionType)]

        #Description should be unique based off ID, so just get first element of list created
        eventMsgDescript = list(eventCode['Event_Msg_Type_Description'].unique())[0]
        actionTypeDescript = list(eventCode['Action_Type_Description'].unique())[0]
        
        #Player who shoots, rebounds, etc
        player = getattr(event, 'Person1')
        
        if verbose:
            print('Counter', counter)
            print('Event Num', eventNum)
            print(skipEventNums)
            print(eventMsgDescript, 'by', player)
    
        counter += 1
        if eventNum in skipEventNums:
            continue
        

        
        teamID = getattr(event, 'Team_id') #teamID doesn't always correspond to player's team
        playerTeam = getPlayerTeam(player, teamID, playerList, gameLineup)
        
        PCtime = getattr(event, 'PC_Time')
        period = getattr(event, 'Period')

     
        if eventMsgDescript == 'Start Period':
            skipEndPeriod = False
            currTime = -1
            currPoss = playerTeam
            #Substitutions at start of period aren't considered substitution events. Must check game linup explicitly.
            team0Lineup = list(gameLineup[(gameLineup['Period'] == period) 
                                          & (gameLineup['Team_id'] == gameTeams[0])]['Person_id'])
            team1Lineup = list(gameLineup[(gameLineup['Period'] == period) 
                                          & (gameLineup['Team_id'] == gameTeams[1])]['Person_id'])

            lastDescript = ''
            
            #Used for cases of a team rebound not assigned to a particular player. The teamID is often wrong, 
            #so when unknownRebound is True, next events are checked to see who has the ball.
            unknownRebound = False
            unknownReboundInfo = {'team0': [], 'team1': [], 'remainingPlayers0': [], 'remainingPlayers1': [], 
                                  'subbedPlayers0': [], 'subbedPlayers1': [], 'event': '', 'lastTeam': ''}
            
            if verbose:
                print(period,' lineup 0:', team0Lineup)
                print(period,' lineup 1:', team1Lineup)
                
                
        elif eventMsgDescript == 'End Period' and not skipEndPeriod:
            if unknownRebound:
                unknownReboundInfo = doPossessionFixSub(unknownReboundInfo, playerTeam, gameDict, gameTeams, verbose)
                unknownRebound = False
                
            #Add possession to each player
            if playerTeam == gameTeams[0]:
                #team0 on offense
                addPossession(team0Lineup, team1Lineup, gameDict)
            else:
                addPossession(team1Lineup, team0Lineup, gameDict)
            lastDescript = ''
            
            if verbose:
                print(period, 'ended')
                
        if lastDescript == 'Missed Shot':
            #Previous event missed shot, check who rebounded
            
            lastDescript = ''
            
            if currTime== 0:
                #Miss at end of period so skip End Period event otherwise double count of possession
                skipEndPeriod = True
                #Add possession to each player
                if lastTeam == gameTeams[0]:
                    #team0 missed on offense
                    addPossession(team0Lineup, team1Lineup, gameDict)
                else:
                    addPossession(team1Lineup, team0Lineup, gameDict)
                continue
                
            currTime = PCtime
            
            if eventMsgDescript != 'Rebound':
                
                #Errors in playbyplay sometimes lead to the wrong order if two things happen at same playclock time
                #Check other events at later time to see if rebound occurs
                
                PCtime = getattr(event, 'PC_Time')
                period = getattr(event, 'Period')
                
                #Get all events after current one and check which events correspond to a rebound in same period
                reboundEvent = gamePlayByPlay[counter-1:]
                reboundEvent = reboundEvent[(reboundEvent['Event_Msg_Type'] == 4) & (reboundEvent['Period'] == period)]
                if reboundEvent.empty:
                    print('No rebound after!')
                    print('Counter', counter-1)
                    print('Event num', getattr(event, 'Event_Num'))
                    break
                #Get first instance of rebound
                player = reboundEvent['Person1'].values[0]
                playerTeam = getPlayerTeam(player, reboundEvent['Team_id'].values[0], playerList, gameLineup)
                
                #Make sure to skip this rebound event
                skipEventNums.append(reboundEvent['Event_Num'].values[0])

            #if player not in lastTeam: #Defensive rebound (for an offensive rebound do nothing)
            
            #Check that if real player (not team rebound) grabbed board, that he is on the court
            #Shouldn't be triggered if parsing done correctly, but good for error checking
            if (player not in team0Lineup) and (player not in team1Lineup) and (player in playerList):
                print('Player not on court', player)
                print('team0', team0Lineup)
                print('team1', team1Lineup)
                print('Counter', counter-1)
                print('Event num', getattr(event, 'Event_Num'))
                break
                
            if player not in playerList:
                #Team rebound, but the teamID might be messed up so don't add possession yet
                unknownRebound = True
                unknownReboundInfo['team0'] = team0Lineup.copy()
                unknownReboundInfo['team1'] = team1Lineup.copy()
                unknownReboundInfo['event'] = 'Missed Shot'
                unknownReboundInfo['lastTeam'] = lastTeam
                    
            elif playerTeam != lastTeam:
                #Defensive rebound
                if playerTeam == gameTeams[0]:
                    #team0 grabs defensive rebound
                    addPossession(team1Lineup, team0Lineup, gameDict)
                    currPoss = gameTeams[0]
                else:
                    addPossession(team0Lineup, team1Lineup, gameDict)
                    currPoss = gameTeams[1]
                if verbose:
                    print("Defensive Rebound")

            else:
                if verbose:
                    print("Offensive Rebound")
            #Reset to player of current event for instances where we needed to find next rebound due to wrong order
            player = getattr(event, 'Person1')
            playerTeam = getPlayerTeam(player, teamID, playerList, gameLineup)
            
        elif lastDescript == 'Missed Last Free Throw':
            #Similar structure as missing last shot, except take into account possibility of substitutions 
            #(important if don't want to give substitutions to player subbed in -- NBA seems to count those)
            lastDescript = ''
            currTime = PCtime
            if eventMsgDescript != 'Rebound':
                #Errors in playbyplay sometimes lead to the wrong order if two things happen at same playclock time
                #Check other events at same time to see if rebound occurs
                
                PCtime = getattr(event, 'PC_Time')
                period = getattr(event, 'Period')
                WCtime = getattr(event, 'WC_Time')
                
                reboundEvent = gamePlayByPlay[counter-1:]
                reboundEvent = reboundEvent[(reboundEvent['Event_Msg_Type'] == 4) & ((reboundEvent['Period'] == period))]
                if reboundEvent.empty:
                    print('Event num', getattr(event, 'Event_Num'))
                    print('No rebound at same time!')
                    break
                #Get first instance of rebound
                player = reboundEvent['Person1'].values[0]
                playerTeam = getPlayerTeam(player, reboundEvent['Team_id'].values[0], playerList, gameLineup)
                
                #Make sure to skip this rebound event
                skipEventNums.append(reboundEvent['Event_Num'].values[0])


            #Check if players subbed at same time
            subEvents = gamePlayByPlay[counter-1:]
            subEvents = subEvents[(subEvents['Event_Msg_Type'] == 8) 
                                & (subEvents['Period'] == period) 
                                & (subEvents['PC_Time'] == PCtime)]
            for substitution in subEvents.itertuples():
                subPlayer = getattr(substitution, 'Person1')
                subEventNum = getattr(substitution, 'Event_Num')
                subPlayerTeam = getPlayerTeam(subPlayer, '', playerList, gameLineup)
                    
                skipEventNums.append(subEventNum)
                doSubstitution(subPlayer, substitution, team0Lineup, team1Lineup, verbose)
                if subPlayerTeam == currPoss:
                    #Player subbed out on offense
                    addPossession([subPlayer], [], gameDict)
                else:
                    addPossession([], [subPlayer], gameDict)
                if verbose:
                    print('Sub after ft')
                    
            if (player not in team0Lineup) and (player not in team1Lineup) and (player in playerList):
                print('Player not on court', player)
                print('team0', team0Lineup)
                print('team1', team1Lineup)
                print('Event num', getattr(event, 'Event_Num'))
                break
            
            subbedPlayers0 = list(set(ftTeam0Lineup) - set(team0Lineup))
            subbedPlayers1 = list(set(ftTeam1Lineup) - set(team1Lineup))
            remainingPlayers0 = list(set(ftTeam0Lineup)-set(subbedPlayers0))
            remainingPlayers1 = list(set(ftTeam1Lineup)-set(subbedPlayers1))
            
            if player not in playerList:
                #Team rebound, but the teamID might be messed up so don't add possession yet
                #UPDATE: I believe NBA counts players being subbed in on a missed last free throw as part of that possession
                #so just add possession to team lineup. Change back to teamLineup back to remainingPlayers if this is undesired
                
                unknownRebound = True
                unknownReboundInfo['team0'] = team0Lineup.copy()
                unknownReboundInfo['team1'] = team1Lineup.copy()
                unknownReboundInfo['event'] = 'Missed Last Free Throw'
                #unknownReboundInfo['remainingPlayers0'] = remainingPlayers0.copy()
                #unknownReboundInfo['remainingPlayers1'] = remainingPlayers1.copy()
            
                unknownReboundInfo['remainingPlayers0'] = team0Lineup.copy()
                unknownReboundInfo['remainingPlayers1'] = team1Lineup.copy()

                unknownReboundInfo['lastTeam'] = lastTeam
                
            #Offensive Rebound, no possessions added (already added to subbed players)
            elif playerTeam == lastTeam:
                if verbose:
                    print("Offensive Rebound")
 
            else:
                #Only add possession to those who were on court when foul occured and are still on court
            
                #UPDATE: I believe NBA counts players being subbed in on a missed last free throw as part of that possession
                #so just add possession to team lineup. Change back to teamLineup back to remainingPlayers if this is undesired
                if playerTeam == gameTeams[0]:
                    #team0 grabs defensive rebound
                    #addPossession(remainingPlayers1, remainingPlayers0, gameDict)
                    addPossession(team1Lineup, team0Lineup, gameDict)
                    currPoss = gameTeams[0]
                else:
                    #addPossession(remainingPlayers0, remainingPlayers1, gameDict)
                    addPossession(team0Lineup, team1Lineup, gameDict)
                    currPoss = gameTeams[1]
                    
                if verbose:
                    print("Defensive Rebound")
                    
            #Reset to player of current event
            player = getattr(event, 'Person1')
            playerTeam = getPlayerTeam(player, teamID, playerList, gameLineup)
        
        if eventNum in skipEventNums:
            #Check this again because of iteration through substitution events after missed shot or free throw
            continue
        
        if eventMsgDescript == 'Substitution':
            doSubstitution(player, event, team0Lineup, team1Lineup, verbose)
            
            #In case current possession unknown use unknownRebound logic to check next possession and assign correct possessions
            if unknownRebound:
                if playerTeam == gameTeams[0]:
                    unknownReboundInfo['subbedPlayers0'].append(player)
                elif playerTeam == gameTeams[1]:
                    unknownReboundInfo['subbedPlayers1'].append(player)
            else:
                if playerTeam == currPoss:
                    #Player subbed out on offense
                    addPossession([player], [], gameDict)
                else:
                    addPossession([], [player], gameDict)
            
        if eventMsgDescript == 'Made Shot':
            currTime = PCtime
            if PCtime == 0:
                skipEndPeriod = True
            
            if unknownRebound:
                unknownReboundInfo = doPossessionFixSub(unknownReboundInfo, playerTeam, gameDict, gameTeams, verbose)
                unknownRebound = False
            
            points = getattr(event, 'Option1')
            
            #Check if and-1. Only checking if fts happen not action type ft 1 out of 1 because possibility of flagrant.
            #This assumes that the ft shot will always be after the made shot in a play by play
            ftEvents = gamePlayByPlay[counter-1:]
            ftEvents = ftEvents[(ftEvents['Event_Msg_Type'] == 3) 
                                    & (ftEvents['Period'] == period) 
                                    & (ftEvents['PC_Time'] == PCtime)]
            # if ft happens then don't count possession, because it will be handled when loop hits that event
            if ftEvents.empty:
                poss = 1
                if playerTeam == gameTeams[0]:
                    currPoss = gameTeams[1]
                else:
                    currPoss = gameTeams[0]
            else:
                poss = 0
            
            addPoints(player,points, team0Lineup, team1Lineup, gameDict, possession= poss)
            
            if verbose:
                print(player, 'scores', points, 'points')
                
        elif eventMsgDescript == 'Turnover':
            
            currTime = PCtime
            
            if unknownRebound:
                unknownReboundInfo = doPossessionFixSub(unknownReboundInfo, playerTeam, gameDict, gameTeams, verbose)
                unknownRebound = False
            
            #Add possession to each player
            if playerTeam == gameTeams[0]:
                #team0 on offense
                addPossession(team0Lineup, team1Lineup, gameDict)
                currPoss = gameTeams[1]
            else:
                addPossession(team1Lineup, team0Lineup, gameDict)
                currPoss = gameTeams[0]
                
            if verbose:
                print(playerTeam, 'turnover')
                
        elif eventMsgDescript == 'Missed Shot':
            
            currTime = PCtime
            if unknownRebound:
                unknownReboundInfo = doPossessionFixSub(unknownReboundInfo, playerTeam, gameDict, gameTeams, verbose)
                unknownRebound = False
            
            #Check who rebounds in next iteration of loop
            lastDescript = eventMsgDescript

            if player in team0Lineup:
                #lastTeam = team0Lineup
                lastTeam = getPlayerTeam(team0Lineup[0], '', playerList, gameLineup)
            else:
                #lastTeam = team1Lineup
                lastTeam = getPlayerTeam(team1Lineup[0], '', playerList, gameLineup)
            if verbose:
                print(player, 'misses shot')

        #Check if Foul (which may or may not lead to free throws), but all free throws come from fouls
        #Important to check if a generic foul and not just shooting fouls, otherwise the free throw lineup will be 'behind'
        #If only looked at shooting fouls - consider situation of substitutions during the game and then a technical foul. 
        #The ft lineup would only be from the previous fts, not the current lineup. 
        elif (eventMsgDescript == 'Foul'):
            #Make sure to copy lineup not assign, otherwise when lineup changes ftTeamLineup would change
            ftTeam0Lineup = team0Lineup.copy()
            ftTeam1Lineup = team1Lineup.copy()
            
            if verbose:
                print('Foul on team', playerTeam)

        elif eventMsgDescript == 'Free Throw':
            if unknownRebound:
                unknownReboundInfo = doPossessionFixSub(unknownReboundInfo, playerTeam, gameDict, gameTeams, verbose)
                unknownRebound = False
            
            option1 = getattr(event, 'Option1')
            if option1 == 1: #Made Free Throw
                points = 1
                if (actionType == 10) or (actionType == 12) or (actionType == 15):
                    #ActionTypes corresponding to final free throw, so add possession. 
                    subbedPlayers0 = list(set(ftTeam0Lineup) - set(team0Lineup))
                    subbedPlayers1 = list(set(ftTeam1Lineup) - set(team1Lineup))
                    
                    #subbedPlayers already got possession added so only add to remaining players on court
                    addPoints(player, points, ftTeam0Lineup, ftTeam1Lineup, gameDict, possession=0)
                    remainingPlayers0 = list(set(ftTeam0Lineup) - set(subbedPlayers0))
                    remainingPlayers1 = list(set(ftTeam1Lineup) - set(subbedPlayers1))
                    
                    if playerTeam == gameTeams[0]:
                        addPossession(remainingPlayers0, remainingPlayers1, gameDict)
                        currPoss = gameTeams[1]
                    elif playerTeam == gameTeams[1]:
                        addPossession(remainingPlayers1, remainingPlayers0, gameDict)
                        currPoss = gameTeams[0]
                    else:
                        print('Error: Free throw player not on either team')
                        return

                    if verbose:
                        print('Made final free throw by', player)
                        
                else:
                    addPoints(player, points, ftTeam0Lineup, ftTeam1Lineup, gameDict, possession=0)
                    
                    if verbose:
                        print('Made free throw by', player)
                        
            else: #Missed FT
                if (actionType == 10) or (actionType == 12) or (actionType == 15):
                    #Missed last free throw for a shooting foul (not technical/flagrant), check who rebounds in next iteration. 
                    lastDescript = 'Missed Last Free Throw'
                    #Offensive rebounds with substitution going to be weird in terms of possessions

                    #Check current lineup, not free throw lineup since current lineup will be the one to rebound
                    if player in team0Lineup:
                        lastTeam = getPlayerTeam(team0Lineup[0], '', playerList, gameLineup)
                    else:
                        lastTeam = getPlayerTeam(team1Lineup[0], '', playerList, gameLineup)
                        
                    if verbose:
                        print('Missed last free throw by', player)
                        print('ftteam1', ftTeam1Lineup)
                else:
                    if verbose:
                        print('Missed free throw by', player)

            #NBA seems to count technical free throws a possession. Delete block below if undesired.
            
            #Check whether was last technical free throw (not flagrant which team keeps possession)
            if (actionType == 16) or (actionType == 22):
                
                
                #Use the possessionFix method already implemented and just count this free throw as the 'unknownRebound'
            
                #So if team on offense commits tech that counts as off + def possession
                #then def + off possession for tech ft, and then following possession will be off + def
                if False:
                    unknownRebound = True
                    unknownReboundInfo['team0'] = team0Lineup.copy()
                    unknownReboundInfo['team1'] = team1Lineup.copy()
                    unknownReboundInfo['event'] = 'Technical Free Throw'
                    unknownReboundInfo['remainingPlayers0'] = ftTeam0Lineup.copy()
                    unknownReboundInfo['remainingPlayers1'] = ftTeam1Lineup.copy()
                    unknownReboundInfo['lastTeam'] = playerTeam
                    
                #Check if tech committed by team with possession. If so add offensive and defensive possession to both teams 
                #(off from having ball, def from tech shot)
                
                #Also it seems like only if time passed in a given possession does the tech count as a change in possession.
                #Remove currTime != PCtime so all techs treated equally
                if currPoss != playerTeam and currTime != PCtime:
                    addPossession(ftTeam1Lineup, ftTeam0Lineup, gameDict)
                    addPossession(ftTeam0Lineup, ftTeam1Lineup, gameDict)
    return gameDict

In [ ]:
dataRows = []
i = 0
for game in GameIDs:
    print(i)
    gamePlayByPlay= sortedPlayByPlay(PlayByPlay[PlayByPlay['Game_id'] == game])
    gameLineup = GameLineup[GameLineup['Game_id'] == game]
    gameDict = gameRatingsSub(gamePlayByPlay, gameLineup, eventCodes)
    playerList = list(gameLineup['Person_id'].unique())
    
    for player in gameDict.keys():
        dataDict = {'Game_ID': game}
        #Check at least one possession played
        offPossessions = gameDict[player][2]
        defPossessions = gameDict[player][3]
        
        totalPossessions = offPossessions + defPossessions
        
        if totalPossessions != 0: 
            #Allowing for possibility of only playing one possession on offense or defense and thus undefined rating
            pointsFor = gameDict[player][0]
            pointsAgainst = gameDict[player][1]
            if offPossessions == 0:
                offRating = np.nan
            else:
                offRating = 100*(pointsFor/offPossessions)
            if defPossessions == 0:
                defRating = np.nan
            else:
                defRating = 100*(pointsAgainst/defPossessions)
            playerTeam = getPlayerTeam(player, '', playerList, gameLineup)
            dataDict.update({'Player_ID': player, 'Team_ID': playerTeam, 'OffRtg': np.round(offRating,1), 
                             'DefRtg': np.round(defRating,1), 'OffPossessions': offPossessions, 
                             'DefPossessions': defPossessions, 'PointsFor': pointsFor, 'PointsAgainst': pointsAgainst})
            
            dataRows.append(dataDict)
    i+=1
df = pd.DataFrame(dataRows)
df= df[['Game_ID', 'Player_ID', 'Team_ID','OffRtg', 'DefRtg', 'OffPossessions', 'DefPossessions', 'PointsFor', 'PointsAgainst']]
df.to_excel("GameRatings.xlsx", index=False)

In [7]:
## bug in playbyplay that cant be fixed? Look at gameIDs[3] events 21-23 show missed free throw, rebound, then made free throw?
#Assuming its even ft, ft, rebound it says it was made so how can there be a rebound?
game = GameIDs[14]
gamePlayByPlay= sortedPlayByPlay(PlayByPlay[PlayByPlay['Game_id'] == game])
gameLineup = GameLineup[GameLineup['Game_id'] == game]
gameDict = gameRatingsSubCleaned(gamePlayByPlay[:], gameLineup, eventCodes, verbose=True)


Counter 0
Event Num 2
[]
Start Period by 0370a0d090da0d0edc6319f120187e0e
1  lineup 0: ['fb64ca4b8beaf4c4c6e4575fe2f3abd7', '7f438c18058290903c46dfe9d71bd68a', '95920e4bf5b6c15ba8dffbf959b38ba5', '722a380c9b59ef42226e8d392824dcb9', 'ef8b068ab7ac9d387b256404acd24cd5']
1  lineup 1: ['3626b893fc73a5cbd67d1ea48a5c7039', '1a6703883f8f47bb4daf09c03be3bda2', '31598ba01a3fff03ed0a87d7dea11dfe', 'a1591595c04d12e88e3cb427fb667618', '6f6a807d57aae8f651222523dc82dc35']
Counter 1
Event Num 4
[]
Jump Ball by 3626b893fc73a5cbd67d1ea48a5c7039
Counter 2
Event Num 7
[]
Missed Shot by 95920e4bf5b6c15ba8dffbf959b38ba5
95920e4bf5b6c15ba8dffbf959b38ba5 misses shot
Counter 3
Event Num 8
[]
Rebound by a1591595c04d12e88e3cb427fb667618
Defensive Rebound
Counter 4
Event Num 9
[]
Turnover by a1591595c04d12e88e3cb427fb667618
cd45058739ed0ac8229849c6249aad48 turnover
Counter 5
Event Num 11
[]
Made Shot by fb64ca4b8beaf4c4c6e4575fe2f3abd7
fb64ca4b8beaf4c4c6e4575fe2f3abd7 scores 2 points
Counter 6
Event Num 12
[]
Mis

2  lineup 1: ['255fe2a8be0ed5c06dd99969ab4fea55', '52c6125836c465f4ac5232121dacb49d', '1a6703883f8f47bb4daf09c03be3bda2', '31598ba01a3fff03ed0a87d7dea11dfe', '821887f9a002be16b5f79729fae59e01']
Counter 96
Event Num 151
[]
Missed Shot by 31598ba01a3fff03ed0a87d7dea11dfe
31598ba01a3fff03ed0a87d7dea11dfe misses shot
Counter 97
Event Num 152
[]
Rebound by 942a84f05f4ab956125f68ec0963481f
Defensive Rebound
Counter 98
Event Num 153
[]
Made Shot by fb64ca4b8beaf4c4c6e4575fe2f3abd7
fb64ca4b8beaf4c4c6e4575fe2f3abd7 scores 2 points
Counter 99
Event Num 154
[]
Foul by 32c044aa84d75ccd78c3c9f2aeb33bd9
Foul on team 8362dfecb7412f12a12713852d2ae566
Counter 100
Event Num 156
[]
Free Throw by 821887f9a002be16b5f79729fae59e01
Made free throw by 821887f9a002be16b5f79729fae59e01
Counter 101
Event Num 157
[]
Free Throw by 821887f9a002be16b5f79729fae59e01
Made final free throw by 821887f9a002be16b5f79729fae59e01
Counter 102
Event Num 158
[]
Turnover by fb64ca4b8beaf4c4c6e4575fe2f3abd7
8362dfecb7412f12a1271

Event Num 284
[]
Free Throw by fb64ca4b8beaf4c4c6e4575fe2f3abd7
Made free throw by fb64ca4b8beaf4c4c6e4575fe2f3abd7
Counter 190
Event Num 285
[]
Free Throw by fb64ca4b8beaf4c4c6e4575fe2f3abd7
Made final free throw by fb64ca4b8beaf4c4c6e4575fe2f3abd7
Counter 191
Event Num 286
[]
Missed Shot by 1a6703883f8f47bb4daf09c03be3bda2
1a6703883f8f47bb4daf09c03be3bda2 misses shot
Counter 192
Event Num 287
[]
Rebound by 7f438c18058290903c46dfe9d71bd68a
Defensive Rebound
Counter 193
Event Num 288
[]
Missed Shot by 7f438c18058290903c46dfe9d71bd68a
7f438c18058290903c46dfe9d71bd68a misses shot
Counter 194
Event Num 289
[]
Rebound by 1a6703883f8f47bb4daf09c03be3bda2
Defensive Rebound
Counter 195
Event Num 290
[]
Missed Shot by 0b978fcfa7f2ec839c563a755e345ff8
0b978fcfa7f2ec839c563a755e345ff8 misses shot
Counter 196
Event Num 291
[]
Rebound by ef8b068ab7ac9d387b256404acd24cd5
Defensive Rebound
Counter 197
Event Num 292
[]
Missed Shot by 1dabb767e07d0aa702ee58d41c15eab1
1dabb767e07d0aa702ee58d41c15eab1 m

95920e4bf5b6c15ba8dffbf959b38ba5 scores 2 points
Counter 284
Event Num 410
[]
Made Shot by 3626b893fc73a5cbd67d1ea48a5c7039
3626b893fc73a5cbd67d1ea48a5c7039 scores 2 points
Counter 285
Event Num 411
[]
Turnover by 1dabb767e07d0aa702ee58d41c15eab1
8362dfecb7412f12a12713852d2ae566 turnover
Counter 286
Event Num 413
[]
Missed Shot by a1591595c04d12e88e3cb427fb667618
a1591595c04d12e88e3cb427fb667618 misses shot
Counter 287
Event Num 414
[]
Rebound by fb64ca4b8beaf4c4c6e4575fe2f3abd7
Defensive Rebound
Counter 288
Event Num 415
[]
Missed Shot by fb64ca4b8beaf4c4c6e4575fe2f3abd7
fb64ca4b8beaf4c4c6e4575fe2f3abd7 misses shot
Counter 289
Event Num 416
[]
Rebound by a1591595c04d12e88e3cb427fb667618
Defensive Rebound
Counter 290
Event Num 417
[]
Made Shot by 31598ba01a3fff03ed0a87d7dea11dfe
31598ba01a3fff03ed0a87d7dea11dfe scores 3 points
Counter 291
Event Num 419
[]
Missed Shot by fb64ca4b8beaf4c4c6e4575fe2f3abd7
fb64ca4b8beaf4c4c6e4575fe2f3abd7 misses shot
Counter 292
Event Num 420
[]
Rebound by

6f6a807d57aae8f651222523dc82dc35 scores 2 points
Counter 365
Event Num 523
[]
Missed Shot by fb64ca4b8beaf4c4c6e4575fe2f3abd7
fb64ca4b8beaf4c4c6e4575fe2f3abd7 misses shot
Counter 366
Event Num 524
[]
Rebound by e49b2cc3f9aacd500b11a35b1c57112d
Offensive Rebound
Counter 367
Event Num 525
[]
Turnover by fb64ca4b8beaf4c4c6e4575fe2f3abd7
8362dfecb7412f12a12713852d2ae566 turnover
Counter 368
Event Num 527
[]
Foul by e49b2cc3f9aacd500b11a35b1c57112d
Foul on team 8362dfecb7412f12a12713852d2ae566
Counter 369
Event Num 529
[]
Substitution by 942a84f05f4ab956125f68ec0963481f
722a380c9b59ef42226e8d392824dcb9  replacing  942a84f05f4ab956125f68ec0963481f
Counter 370
Event Num 539
[]
Substitution by e49b2cc3f9aacd500b11a35b1c57112d
95920e4bf5b6c15ba8dffbf959b38ba5  replacing  e49b2cc3f9aacd500b11a35b1c57112d
Counter 371
Event Num 533
[]
Turnover by 1a6703883f8f47bb4daf09c03be3bda2
cd45058739ed0ac8229849c6249aad48 turnover
Counter 372
Event Num 534
[]
Missed Shot by 95920e4bf5b6c15ba8dffbf959b38ba5
9

a1591595c04d12e88e3cb427fb667618 scores 3 points
Counter 449
Event Num 643
[557]
Missed Shot by fb64ca4b8beaf4c4c6e4575fe2f3abd7
fb64ca4b8beaf4c4c6e4575fe2f3abd7 misses shot
Counter 450
Event Num 644
[557]
Rebound by fb64ca4b8beaf4c4c6e4575fe2f3abd7
Offensive Rebound
Counter 451
Event Num 645
[557]
Foul by a1591595c04d12e88e3cb427fb667618
Foul on team cd45058739ed0ac8229849c6249aad48
Counter 452
Event Num 647
[557]
Free Throw by fb64ca4b8beaf4c4c6e4575fe2f3abd7
Made free throw by fb64ca4b8beaf4c4c6e4575fe2f3abd7
Counter 453
Event Num 648
[557]
Substitution by 95920e4bf5b6c15ba8dffbf959b38ba5
ef8b068ab7ac9d387b256404acd24cd5  replacing  95920e4bf5b6c15ba8dffbf959b38ba5
Counter 454
Event Num 650
[557]
Free Throw by fb64ca4b8beaf4c4c6e4575fe2f3abd7
Made final free throw by fb64ca4b8beaf4c4c6e4575fe2f3abd7
Counter 455
Event Num 651
[557]
Made Shot by 31598ba01a3fff03ed0a87d7dea11dfe
31598ba01a3fff03ed0a87d7dea11dfe scores 3 points
Counter 456
Event Num 653
[557]
Missed Shot by fb64ca4b8bea

In [8]:
gameDict

{'0b978fcfa7f2ec839c563a755e345ff8': array([29, 20, 22, 23]),
 '14de632b07100527b0ced12fd4eeffb7': array([0, 0, 0, 0]),
 '18a823379f2bf4e4be8b419698cde91c': array([0, 0, 0, 0]),
 '1a6703883f8f47bb4daf09c03be3bda2': array([113, 103,  86,  87]),
 '1dabb767e07d0aa702ee58d41c15eab1': array([73, 78, 69, 67]),
 '255fe2a8be0ed5c06dd99969ab4fea55': array([ 6, 12,  9,  8]),
 '31598ba01a3fff03ed0a87d7dea11dfe': array([108,  99,  85,  86]),
 '32c044aa84d75ccd78c3c9f2aeb33bd9': array([41, 29, 32, 31]),
 '3626b893fc73a5cbd67d1ea48a5c7039': array([113,  96,  86,  87]),
 '36fdadf436b164ee29174c8e1fde7271': array([0, 0, 1, 0]),
 '3d75035d20b173a867d4bf32c8a58f0b': array([29, 27, 25, 26]),
 '52c6125836c465f4ac5232121dacb49d': array([39, 36, 36, 35]),
 '619d3e44dc84b366bd685de3e94b3bec': array([0, 0, 1, 0]),
 '6f6a807d57aae8f651222523dc82dc35': array([49, 59, 47, 49]),
 '722a380c9b59ef42226e8d392824dcb9': array([67, 78, 55, 55]),
 '7f438c18058290903c46dfe9d71bd68a': array([76, 98, 75, 73]),
 '821887f9a0

In [9]:
rdPBP = readablePlayByPlay(gamePlayByPlay, gameLineup, eventCodes)

Team0 8362dfecb7412f12a12713852d2ae566
Team1 cd45058739ed0ac8229849c6249aad48


In [10]:
rdPBP[:]

,Game_id,Event_Num,Event_Msg_Type,Period,WC_Time,PC_Time,Action_Type,Option1,Option2,Option3,Team_id,Person1,Person2,Person3,Team_id_type,Person1_type,Person2_type,Person3_type
6315,3ce947db2df86b08a40b7526e2faaccb,2,Start Period,1,618770,7200,,0,0,0,Officials,0370a0d090da0d0edc6319f120187e0e,0370a0d090da0d0edc6319f120187e0e,0370a0d090da0d0edc6319f120187e0e,0,0,0,0
6316,3ce947db2df86b08a40b7526e2faaccb,4,Jump Ball,1,618770,7200,,0,0,0,Team1,3626b893fc73a5cbd67d1ea48a5c7039,ef8b068ab7ac9d387b256404acd24cd5,95920e4bf5b6c15ba8dffbf959b38ba5,3,4,5,5
6317,3ce947db2df86b08a40b7526e2faaccb,7,Missed Shot,1,618960,7010,Hook Shot,2,0,0,Team0,95920e4bf5b6c15ba8dffbf959b38ba5,0370a0d090da0d0edc6319f120187e0e,0370a0d090da0d0edc6319f120187e0e,3,5,0,0
6318,3ce947db2df86b08a40b7526e2faaccb,8,Rebound,1,619000,7000,Unknown,0,0,0,Team1,a1591595c04d12e88e3cb427fb667618,0370a0d090da0d0edc6319f120187e0e,0370a0d090da0d0edc6319f120187e0e,3,4,0,0
6319,3ce947db2df86b08a40b7526e2faaccb,9,Turnover,1,619040,6930,Bad Pass,1,0,0,Team1,a1591595c04d12e88e3cb427fb667618,fb64ca4b8beaf4c4c6e4575fe2f3abd7,0370a0d090da0d0edc6319f120187e0e,2,4,5,0
6320,3ce947db2df86b08a40b7526e2faaccb,11,Made Shot,1,619270,6700,Driving Finger Roll Layup Shot,2,0,0,Team0,fb64ca4b8beaf4c4c6e4575fe2f3abd7,0370a0d090da0d0edc6319f120187e0e,0370a0d090da0d0edc6319f120187e0e,3,5,0,0
6321,3ce947db2df86b08a40b7526e2faaccb,12,Missed Shot,1,619470,6500,Turnaround Fadeaway shot,2,0,0,Team1,3626b893fc73a5cbd67d1ea48a5c7039,0370a0d090da0d0edc6319f120187e0e,0370a0d090da0d0edc6319f120187e0e,2,4,0,0
6322,3ce947db2df86b08a40b7526e2faaccb,13,Rebound,1,619520,6490,Unknown,0,0,0,Team0,95920e4bf5b6c15ba8dffbf959b38ba5,0370a0d090da0d0edc6319f120187e0e,0370a0d090da0d0edc6319f120187e0e,2,5,0,0
6323,3ce947db2df86b08a40b7526e2faaccb,14,Made Shot,1,619600,6370,Layup Shot,2,0,0,Team0,7f438c18058290903c46dfe9d71bd68a,0370a0d090da0d0edc6319f120187e0e,0370a0d090da0d0edc6319f120187e0e,3,5,0,0
6324,3ce947db2df86b08a40b7526e2faaccb,15,Missed Shot,1,619720,6290,Jump Shot,3,0,0,Team1,31598ba01a3fff03ed0a87d7dea11dfe,0370a0d090da0d0edc6319f120187e0e,0370a0d090da0d0edc6319f120187e0e,2,4,0,0


In [257]:
rdPBP[287:331]

,Game_id,Event_Num,Event_Msg_Type,Period,WC_Time,PC_Time,Action_Type,Option1,Option2,Option3,Team_id,Person1,Person2,Person3,Team_id_type,Person1_type,Person2_type,Person3_type
32873,dc3a96d83623f28fd84bd535e57c146f,406,Missed Shot,3,639810,3810,Jump Shot,3,0,0,Team0,8c646f2710d4f64955903039fdc6374e,0370a0d090da0d0edc6319f120187e0e,0370a0d090da0d0edc6319f120187e0e,3,5,0,0
32874,dc3a96d83623f28fd84bd535e57c146f,407,Rebound,3,639840,3800,Unknown,0,0,0,Team1,95c8abc40fb4ecf4fa5627975450dc95,0370a0d090da0d0edc6319f120187e0e,0370a0d090da0d0edc6319f120187e0e,3,4,0,0
32875,dc3a96d83623f28fd84bd535e57c146f,408,Foul,3,639860,3790,Personal,0,0,0,Team0,b43d8e250bbf5bf03bcbbd2f48141acd,95c8abc40fb4ecf4fa5627975450dc95,4be6e006a878ead702fec2abcf8b904a,2,5,4,1
32876,dc3a96d83623f28fd84bd535e57c146f,410,Substitution,3,639970,3790,,0,0,0,Team0,4b35bf5cc8eebb940aa41c466bff7277,e9b4d02c8216e149e1b0c69c9feb6541,0370a0d090da0d0edc6319f120187e0e,2,5,5,0
32877,dc3a96d83623f28fd84bd535e57c146f,413,Foul,3,640330,3640,Shooting,0,0,2,Team0,8c646f2710d4f64955903039fdc6374e,b265ffd84dbeef427136d140811e7d5f,227d578f7628770519b586873c704e07,2,5,4,1
32878,dc3a96d83623f28fd84bd535e57c146f,415,Foul,3,640670,3640,Technical,0,0,1,Team0,8c646f2710d4f64955903039fdc6374e,0370a0d090da0d0edc6319f120187e0e,4be6e006a878ead702fec2abcf8b904a,2,5,0,1
32879,dc3a96d83623f28fd84bd535e57c146f,416,Free Throw,3,640760,3640,Free Throw Technical,1,0,0,Team1,95c8abc40fb4ecf4fa5627975450dc95,0370a0d090da0d0edc6319f120187e0e,0370a0d090da0d0edc6319f120187e0e,2,4,0,0
32880,dc3a96d83623f28fd84bd535e57c146f,417,Free Throw,3,640970,3640,Free Throw 1 of 2,1,0,0,Team1,b265ffd84dbeef427136d140811e7d5f,0370a0d090da0d0edc6319f120187e0e,0370a0d090da0d0edc6319f120187e0e,2,4,0,0
32881,dc3a96d83623f28fd84bd535e57c146f,418,Free Throw,3,641130,3640,Free Throw 2 of 2,1,0,0,Team1,b265ffd84dbeef427136d140811e7d5f,0370a0d090da0d0edc6319f120187e0e,0370a0d090da0d0edc6319f120187e0e,2,4,0,0
32882,dc3a96d83623f28fd84bd535e57c146f,419,Timeout,3,641230,3640,Regular,0,0,0,Team0,f3d9bebad56898206207ca7cc07323f7,0370a0d090da0d0edc6319f120187e0e,0370a0d090da0d0edc6319f120187e0e,3,3,0,0


In [116]:
np.round(100*21.0/16, 2)

131.25

In [172]:
rdPBPsmall = rdPBP[419:424]
rdPBPsmall

,Game_id,Event_Num,Event_Msg_Type,Period,WC_Time,PC_Time,Action_Type,Option1,Option2,Option3,Team_id,Person1,Person2,Person3,Team_id_type,Person1_type,Person2_type,Person3_type
25659,bb4e1b593cc210dec5fdcb0f7eb4739d,609,Missed Shot,4,557230,1970,Floating Jump shot,2,0,0,Team0,ff52c317e26534ae1679da3c917e9fec,0370a0d090da0d0edc6319f120187e0e,0370a0d090da0d0edc6319f120187e0e,2,4,0,0
25661,bb4e1b593cc210dec5fdcb0f7eb4739d,610,Rebound,4,557280,1920,Unknown,0,0,0,Team0,83c15c0962941640faab838a8f6f151d,0370a0d090da0d0edc6319f120187e0e,0370a0d090da0d0edc6319f120187e0e,2,4,0,0
25660,bb4e1b593cc210dec5fdcb0f7eb4739d,611,Made Shot,4,557280,1920,Putback Dunk Shot,2,0,0,Team0,83c15c0962941640faab838a8f6f151d,0370a0d090da0d0edc6319f120187e0e,0370a0d090da0d0edc6319f120187e0e,2,4,0,0
25662,bb4e1b593cc210dec5fdcb0f7eb4739d,612,Foul,4,557350,1880,Shooting,0,0,2,Team0,7dfbb5980c066844384ba7424aceae47,f562f72f83b8e06b60ff184531b07056,5201a0c1055bebf092c5ce178e5c7863,3,4,5,1
25663,bb4e1b593cc210dec5fdcb0f7eb4739d,614,Free Throw,4,557590,1880,Free Throw 1 of 2,1,0,0,Team1,f562f72f83b8e06b60ff184531b07056,0370a0d090da0d0edc6319f120187e0e,0370a0d090da0d0edc6319f120187e0e,3,5,0,0


In [11]:
for i in xrange(1,5):
    print(i)

NameError: name 'xrange' is not defined

In [182]:
ser = (rdPBPsmall['Event_Num'] == rdPBPsmallReb['Event_Num'])
np.where(ser.values == True)[0][0]

1

In [186]:
for p in rdPBPsmall[5:].itertuples():
    print('d')

In [307]:
i = np.where(GameIDs=='bb4e1b593cc210dec5fdcb0f7eb4739d')
print (i, GameIDs[i])

(array([55], dtype=int64),) ['bb4e1b593cc210dec5fdcb0f7eb4739d']


array(['97b8982d363b55efc61eae5d969d7f44'], dtype=object)